In [20]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [56]:
ford_bs = pd.read_csv("C:/Users/Gast01/Desktop/Master Thesis Finance/Balance Sheets/Ford bs.csv",  on_bad_lines='skip')
ford_stock = pd.read_csv("C:/Users/Gast01/Desktop/Master Thesis Finance/Stock Prices/Ford Motor Stock Price History.csv")
market_stock = pd.read_csv("C:/Users/Gast01/Desktop/Master Thesis Finance/Stock Prices/S&P 500 Historical Data.csv")

In [57]:
ford_melted = ford_bs.melt(id_vars=["Breakdown"], var_name="Date", value_name="Value")
required_items = ["Total Assets", "Total Liabilities Net Minority Interest", 'Cash', 'Cash Equivalents', 'Total Capitalization']
ford_filtered = ford_melted[ford_melted["Breakdown"].isin(required_items)]
ford_df = ford_filtered.pivot(index="Date", columns="Breakdown", values="Value").reset_index()
ford_df.columns.name = None  # Remove column grouping
ford_df.rename(columns={"Date": "Date"}, inplace=True)  # Keep Date explicit

ford_df["Net Income"] = pd.Series([4056000, 2536000, 4235000, 3409000, 201000]).astype(int)
ford_df["Cash"] = ford_df["Cash"] + ford_df["Cash Equivalents"]
ford_df = ford_df.drop(columns="Cash Equivalents")

In [58]:
ford_stock = ford_stock.iloc[::-1].reset_index(drop=True)

ford_stock["Date"] = pd.to_datetime(ford_stock["Date"])

ford_stock = ford_stock.set_index("Date").asfreq("D")  # "D" ensures daily frequency

ford_stock = ford_stock.ffill()

ford_stock = ford_stock.reset_index()

ford_stock

,Date,Price,Open,High,Low,Vol.,Change %
0,2023-10-02,12.13,12.20,12.26,12.05,51.20M,-0.89%
1,2023-10-03,11.90,11.99,12.05,11.85,55.07M,-1.95%
2,2023-10-04,11.82,11.85,11.91,11.71,45.22M,-0.66%
3,2023-10-05,11.73,11.77,11.83,11.52,47.43M,-0.75%
4,2023-10-06,11.83,11.65,11.98,11.52,57.56M,0.84%
...,...,...,...,...,...,...,...
452,2024-12-27,10.03,10.02,10.20,9.98,52.90M,-0.40%
453,2024-12-28,10.03,10.02,10.20,9.98,52.90M,-0.40%
454,2024-12-29,10.03,10.02,10.20,9.98,52.90M,-0.40%
455,2024-12-30,9.88,9.95,9.98,9.82,47.12M,-1.50%


In [59]:
market_stock = market_stock.iloc[::-1].reset_index(drop=True)

market_stock["Date"] = pd.to_datetime(market_stock["Date"])

market_stock = market_stock.set_index("Date").asfreq("D")  # "D" ensures daily frequency

market_stock = market_stock.ffill()

market_stock = market_stock.reset_index()

market_stock["Price"] = market_stock["Price"].replace({',': ''}, regex=True).astype(float)

market_stock

,Date,Price,Open,High,Low,Vol.,Change %
0,2023-10-02,4288.39,"4,284.52","4,300.58","4,260.21",NaN,0.01%
1,2023-10-03,4229.45,"4,269.75","4,281.15","4,216.45",NaN,-1.37%
2,2023-10-04,4263.75,"4,233.83","4,268.50","4,220.48",NaN,0.81%
3,2023-10-05,4258.19,"4,259.31","4,267.13","4,225.91",NaN,-0.13%
4,2023-10-06,4308.50,"4,234.79","4,324.10","4,219.55",NaN,1.18%
...,...,...,...,...,...,...,...
452,2024-12-27,5970.84,"6,006.17","6,006.17","5,932.95",NaN,-1.11%
453,2024-12-28,5970.84,"6,006.17","6,006.17","5,932.95",NaN,-1.11%
454,2024-12-29,5970.84,"6,006.17","6,006.17","5,932.95",NaN,-1.11%
455,2024-12-30,5906.94,"5,920.67","5,940.79","5,869.16",NaN,-1.07%


In [60]:
ford_df["D2A"] = ford_df["Total Liabilities Net Minority Interest"] / ford_df["Total Assets"]
ford_df["NI2A"] = ford_df["Net Income"] / ford_df["Total Assets"]
ford_df["C2A"] = ford_df["Cash"] / ford_df["Total Assets"]
ford_df["E2D"] = ford_df["Total Capitalization"] / ford_df["Total Liabilities Net Minority Interest"]

In [61]:
ford_stock = ford_stock.iloc[::-1].reset_index(drop=True)
ford_stock["Stock Returns"] = np.log(ford_stock["Price"] / ford_stock["Price"].shift(1))

In [62]:
market_stock = market_stock.iloc[::-1].reset_index(drop=True)
market_stock["Market Returns"] = np.log(market_stock["Price"]/ market_stock["Price"].shift(1))

In [63]:
ford_stock

,Date,Price,Open,High,Low,Vol.,Change %,Stock Returns
0,2024-12-31,9.90,9.91,10.01,9.84,54.10M,0.20%,NaN
1,2024-12-30,9.88,9.95,9.98,9.82,47.12M,-1.50%,-0.002022
2,2024-12-29,10.03,10.02,10.20,9.98,52.90M,-0.40%,0.015068
3,2024-12-28,10.03,10.02,10.20,9.98,52.90M,-0.40%,0.000000
4,2024-12-27,10.03,10.02,10.20,9.98,52.90M,-0.40%,0.000000
...,...,...,...,...,...,...,...,...
452,2023-10-06,11.83,11.65,11.98,11.52,57.56M,0.84%,0.000000
453,2023-10-05,11.73,11.77,11.83,11.52,47.43M,-0.75%,-0.008489
454,2023-10-04,11.82,11.85,11.91,11.71,45.22M,-0.66%,0.007643
455,2023-10-03,11.90,11.99,12.05,11.85,55.07M,-1.95%,0.006745


In [64]:
ford_stock["Stock Sigma"] = ford_stock["Stock Returns"].rolling(window=60).std()

market_stock["Stock Sigma"] = market_stock["Market Returns"].rolling(window=60).std()

In [67]:
ford_stock["EXCESS_RETURN"] = ford_stock["Stock Returns"] - market_stock["Market Returns"]

In [70]:
stock_data = ford_stock[["Date", "Stock Sigma", "EXCESS_RETURN"]].dropna()

stock_data

,Date,Stock Sigma,EXCESS_RETURN
60,2024-11-01,0.014910,0.000000
61,2024-10-31,0.014928,0.010910
62,2024-10-30,0.014968,-0.001449
63,2024-10-29,0.014991,-0.009053
64,2024-10-28,0.018777,0.089824
...,...,...,...
452,2023-10-06,0.023461,0.000000
453,2023-10-05,0.023479,0.003257
454,2023-10-04,0.023427,0.006338
455,2023-10-03,0.023433,0.014822


In [74]:
ford_df["Date"] = pd.to_datetime(ford_df["Date"])
ford_stock["Date"] = pd.to_datetime(ford_stock["Date"])

ford_stock = ford_stock.sort_values(by="Date")

ford_df = pd.merge_asof(ford_df.sort_values("Date"), ford_stock[["Date", "Price", "EXCESS_RETURN", "Stock Sigma"]], 
                        on="Date", direction="backward")

In [89]:
def label_distress(row):
    if row['D2A'] > 0.6 and row['NI2A'] < 0.01:  # Example conditions
        return 1  # Default (Distress)
    else:
        return 0  # Non-default (Non-distress)

# Apply the function to create a binary target variable
ford_df['Default'] = ford_df.apply(label_distress, axis=1)

In [99]:
ford_df

,Date,Cash,Total Assets,Total Capitalization,Total Liabilities Net Minority Interest,Net Income,D2A,NI2A,C2A,E2D,Price,EXCESS_RETURN,Stock Sigma,Default
0,2023-12-31,24862000.0,273310000.0,142335000.0,230512000.0,4056000,0.843409,0.014840,0.090966,0.617473,12.02,0.000000,0.017277,0
1,2024-03-31,19721000.0,274341000.0,142495000.0,231444000.0,4235000,0.843636,0.015437,0.071885,0.615678,13.28,-0.002768,0.016732,0
2,2024-06-30,19953000.0,276586000.0,143903000.0,232991000.0,3409000,0.842382,0.012325,0.072140,0.617633,12.54,-0.014720,0.031354,0
3,2024-09-30,23449000.0,287047000.0,148009000.0,242708000.0,201000,0.845534,0.000700,0.081690,0.609823,10.56,-0.027200,0.018770,1
4,2024-12-31,22935000.0,285196000.0,148408000.0,240338000.0,2536000,0.842712,0.008892,0.080418,0.617497,9.90,NaN,NaN,1


In [104]:
features = ["D2A", "NI2A", "C2A", "E2D"]
data = ford_df[features]

X = data[features]
y = ford_df["Default"]

In [105]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [106]:
logit_model = LogisticRegression()
logit_model.fit(X_scaled, y)

LogisticRegression()

In [107]:
latest_data = data.iloc[-1:][features]
latest_scaled = scaler.transform(latest_data)

pd_estimate = logit_model.predict_proba(latest_scaled)[:, 1]

print(f"Estimated Probability of Default: {pd_estimate[0]:.4f}")

Estimated Probability of Default: 0.4209
